## Predicting Iowa Housing Prices Using the Random Forests Algorithm

In [2]:
import pandas as pd
import pprint
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


# Path of the file to read
iowa_file_path = 'home-data/train.csv'
home_data = pd.read_csv(iowa_file_path)

### Let's create a target object and call it 'y' and X that contains the features that we want to consider in building our model.

In [3]:
# Create target object and call it y
y = home_data.SalePrice

# Create X
features = ['LotArea', 'YearRemodAdd','YearBuilt','YrSold','GarageCars','KitchenAbvGr','1stFlrSF','2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

### Let's split our dataset into validation and training data using train_test_split. Afterwards, we can now build our model by specifiying what kind of ML algorithm we want to employ. For this case, we will use DecisionTreeRegressor to build our model based on the Decision Tree learning method.

### After building, we will feed our training dataset to the model.

In [6]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


# Random forests model
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)



/Users/kimberlyroselagrama/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

### Now we can validate our model by generating the predicting values and determining the mean_absolute_error

In [7]:
iowa_predict = forest_model.predict(val_X)
rf_val_mae = mean_absolute_error(val_y, iowa_predict)
print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21724.227397260274
